In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

# Path to the SQLite database
db_path = Path("../telco_churn.db")

# Connect to the database
conn = sqlite3.connect(db_path)

In [2]:
pd.read_sql_query(
    """
    SELECT
      AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) AS churn_rate
    FROM telco_churn;
    """ ,
    conn
)

,churn_rate
0,0.265785


In [3]:
pd.read_sql_query(
    """
    SELECT
      CASE
        WHEN tenure <= 12 THEN '0–12'
        WHEN tenure <= 24 THEN '12–24'
        WHEN tenure <= 36 THEN '24–36'
        WHEN tenure <= 48 THEN '36–48'
        WHEN tenure <= 60 THEN '48–60'
        ELSE '60+'
      END AS TenureGroup,
      AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) AS churn_rate
    FROM telco_churn
    GROUP BY TenureGroup
    ORDER BY
      CASE TenureGroup
        WHEN '0–12' THEN 1
        WHEN '12–24' THEN 2
        WHEN '24–36' THEN 3
        WHEN '36–48' THEN 4
        WHEN '48–60' THEN 5
        ELSE 6
      END;
    """,
    conn
)

,TenureGroup,churn_rate
0,0–12,0.476782
1,12–24,0.287109
2,24–36,0.216346
3,36–48,0.190289
4,48–60,0.144231
5,60+,0.066098


In [4]:
query = """
SELECT
  Contract,
  CASE
    WHEN tenure <= 12 THEN '0–12'
    WHEN tenure <= 24 THEN '12–24'
    WHEN tenure <= 36 THEN '24–36'
    WHEN tenure <= 48 THEN '36–48'
    WHEN tenure <= 60 THEN '48–60'
    ELSE '60+'
  END AS TenureGroup,
  AVG(CASE WHEN Churn = 'Yes' THEN 1.0 ELSE 0.0 END) AS churn_rate
FROM telco_churn
GROUP BY Contract, TenureGroup
ORDER BY Contract, TenureGroup;
"""

pd.read_sql_query(query, conn)

,Contract,TenureGroup,churn_rate
0,Month-to-month,0–12,0.513541
1,Month-to-month,12–24,0.377205
2,Month-to-month,24–36,0.325103
3,Month-to-month,36–48,0.335443
4,Month-to-month,48–60,0.277778
5,Month-to-month,60+,0.222222
6,One year,0–12,0.105691
7,One year,12–24,0.081218
8,One year,24–36,0.080000
9,One year,36–48,0.130597


In [5]:
conn.close()

## SQL validation
The SQL-based churn KPIs match the results obtained in pandas, confirming that the analysis is consistent across tools and could be reproduced in a database environment.